<a href="https://colab.research.google.com/github/jimmy-io/Coursera_Capstone/blob/master/LA_food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Capstone Project

## Jimmy J.

### Introduction
Frank Lloyd Wright said it best - “Tip the world over on its side and everything loose will land in Los Angeles." For decades people from all over the world have come to this city in search of home, family, love, fame, and fortune. Los Angeles welcomes this multicultural migration with a suburban sprawl that encompases over 88 cities, and even more unincorporated neihborhoods. Each of these neighborhoods are characterized by geographical, economic, and cultural features that make them uniquely poised to cater to different demographics.
Neighborhoods like Downtown Los Angeles, Culver City, Long Beach, Century City, and West Hollywood provide a mixture of urban style living and accessibility to grocery stores, malls, means of public transportation and entertainment venues within walking distance. On the otherhand suburbs like Baldwin Hills, Crenshaw, Echo Park and Boyle Heights are quiter neighborhoods with single family dwellings that are not easily accessible via public modes of transportation.
Given the vast spectrum of possibilites of neighborhoods to choose from in LA, someone looking to move here might be overwhelmed. In this project I've attempted to characterize neighborhoods in LA by the nature of venue that are in their immediate vicinities using a clustering algorithm. Results from this analysis show that neighborhoods in LA fall under a few groups, defined by the nature of the venues closest to them. This is of most interest to rental unit searching apps like Westiside Rentals or Rentpad, to real estate agents, and generally, to people looking to move to LA. The results of this project can help them find neighborhoods that are most aligned with what they are looking for in a place to live and overall, provide a more satisfactory experience than chosing a neighborhood at random.

### Data
The list of neighborhoods in Los Angeles was web scraped from Wikipedia using BeautifulSoup. The names of these neigborhoods were then fed into Nominatim to obtain their geographical coordinates.
The venues in the vicinity of these neighborhoods will be retreived using the FourSquare API search engine. Venues of five different categories were chosen: Travel & Transport, Arts & Entertainment, Outdoors & Recreation, Nightlife Spot, and Food. The number of venues of each category for each neighborhood was counted and then normalized to give five parameters with which to cluster the neighborhoods by.

In [0]:
### Importing libraries 

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import re
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import bs4 as bs
import urllib.request
import requests
import collections
import csv

In [4]:
#@hidden_cell

CLIENT_ID = 'JS0P2BHNS4GICN4OT1LRM03JV0OLTO4QWS0I5AEITRLVI3QU' # your Foursquare ID
CLIENT_SECRET = 'KGFP21SEFHLUXM2EPAI4HDLQOAI21MC1CY24RJ4AII4UX2Q3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails: hidden')

Your credentails: hidden


In [5]:
# Retreiving the coordinates for Los Angeles 

address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="LA_explorer")
location = geolocator.geocode(address)
LAlatitude = location.latitude
LAlongitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(LAlatitude, LAlongitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.


In [6]:
## Categories and IDs from FourSquare API

catIds={}
catIds={
        'Food':'4d4b7105d754a06374d81259',
 
       }
catIds

{'Food': '4d4b7105d754a06374d81259'}

In [7]:
for key in catIds:
    print(key)
    print(catIds[key])


Food
4d4b7105d754a06374d81259


In [0]:
### Foursqaure API

venues_list=[]
offset = 50
LIMIT = 10000 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
categoryId='4d4b7105d754a06374d81259'

for o in range(offset):
  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}&offset={}'.format(CLIENT_ID, 
                                                                                                                                              CLIENT_SECRET, 
                                                                                                                                              VERSION, 
                                                                                                                                              LAlatitude,
                                                                                                                                              LAlongitude, 
                                                                                                                                              radius,
                                                                                                                                              categoryId,
                                                                                                                                              LIMIT,
                                                                                                                                              o)
            
    # make the GET request
            
  results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
    # return only relevant information for each nearby venue
  venues_list.append([(v['venue']['id'],
                     v['venue']['name'], 
                      v['venue']['location']['lat'], 
                      v['venue']['location']['lng'],  
                      v['venue']['categories'][0]['name']) for v in results])
            
  nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
  nearby_venues.columns = ['id',
                         'Venue', 
                          'Venue Latitude', 
                          'Venue Longitude', 
                          'Venue Category']
        



In [9]:
nearby_venues

,id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,54938133498ed65f02e8c4ba,Redbird,34.050666,-118.244068,American Restaurant
1,5143f2d7e4b039102cf9793f,Marugame Monzo,34.049807,-118.240202,Udon Restaurant
2,53d06bba498e6c801ea9816e,KazuNori: The Original Hand Roll Bar,34.047716,-118.247452,Sushi Restaurant
3,5956f0d0f5e9d7161f043456,Sari Sari Store LA,34.051065,-118.249390,Filipino Restaurant
4,4da117b2bf22a143f96894f8,Café Dulcé,34.048869,-118.240508,Bakery
...,...,...,...,...,...
4995,4b367855f964a520353625e3,Square One Dining,34.095356,-118.295069,Breakfast Spot
4996,54d17a8e498e41a7ccdc3471,Taqueria el zarape,34.095242,-118.302558,Mexican Restaurant
4997,4f9ae708e4b0ad3854b71528,Lemon Poppy Kitchen,34.112303,-118.235532,Café
4998,53ae12ee498e1ac3c126d0be,H Coffee House,34.102927,-118.287316,Café


In [10]:
group = nearby_venues.groupby('Venue Category', sort = True).count().sort_values('id', ascending = False)

group

,id,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,
Korean Restaurant,541,541,541,541
Mexican Restaurant,333,333,333,333
Café,304,304,304,304
Food Truck,273,273,273,273
American Restaurant,267,267,267,267
Taco Place,263,263,263,263
New American Restaurant,232,232,232,232
Asian Restaurant,228,228,228,228
Burger Joint,177,177,177,177


In [0]:
## Saving the nearby venues data as a csv to avoid repeated API calls 
nearby_venues.to_csv('/content/drive/My Drive/IBM Capstone/LA_food.csv')

In [0]:
## Reading the nearby venues data as a csv
la_food  = pd.read_csv('/content/drive/My Drive/IBM Capstone/LA_food.csv', index_col=[0])

In [13]:
la_food.tail(10)

,id,Venue,Venue Latitude,Venue Longitude,Venue Category
4990,4ce0287b825e721ed2cd7445,Proof Bakery,34.116978,-118.261641,Bakery
4991,5606e561498ecd0d68963621,Pasta Sisters,34.047503,-118.317216,Italian Restaurant
4992,54c7e947498e0337af1a23c0,Dune,34.116959,-118.262060,Mediterranean Restaurant
4993,4ac0250ef964a520f39320e3,Noshi Sushi,34.076159,-118.305374,Sushi Restaurant
4994,4ac41be2f964a520399e20e3,Osteria La Buca,34.083420,-118.313920,Italian Restaurant
4995,4b367855f964a520353625e3,Square One Dining,34.095356,-118.295069,Breakfast Spot
4996,54d17a8e498e41a7ccdc3471,Taqueria el zarape,34.095242,-118.302558,Mexican Restaurant
4997,4f9ae708e4b0ad3854b71528,Lemon Poppy Kitchen,34.112303,-118.235532,Café
4998,53ae12ee498e1ac3c126d0be,H Coffee House,34.102927,-118.287316,Café
4999,4ce2dd06825e721e11808345,La Azteca Tortilleria,34.040503,-118.168017,Latin American Restaurant


In [0]:
## Yelp API

api_key='9gKVy_2yMKuewa8S6df3lV2EFmjpZLGUGyWG9g5z72H7DGEQCqHnCS0v3yU98xTbignxd84d-3KM1_waOnHIXDBmVkGg_fxGTdDe9850aCkERzRAh9KkZc_P8xJbXnYx'
headers = {'Authorization': 'Bearer %s' % api_key}

url='https://api.yelp.com/v3/businesses/search'

# In the dictionary, term can take values like food, cafes or businesses like McDonalds
params = {'term':'Yang san bak','location':'Los Angeles'}

# Making a get request to the API
req=requests.get(url, params=params, headers=headers)

rest_id = json.loads(req.text)['businesses'][0]['id']

url='https://api.yelp.com/v3/businesses/{}'.format(rest_id)

req1=requests.get(url, headers=headers)

rest_json=json.loads(req1.text)

In [15]:
features_list=[]

features_list.extend((rest_json['alias'],
                     rest_json['categories']))

features_list

['yang-san-bak-los-angeles',
 [{'alias': 'korean', 'title': 'Korean'},
  {'alias': 'bbq', 'title': 'Barbeque'}]]

In [16]:
rest_json

{'alias': 'yang-san-bak-los-angeles',
 'categories': [{'alias': 'korean', 'title': 'Korean'},
  {'alias': 'bbq', 'title': 'Barbeque'}],
 'coordinates': {'latitude': 34.06388, 'longitude': -118.30073},
 'display_phone': '(213) 365-9689',
 'hours': [{'hours_type': 'REGULAR',
   'is_open_now': True,
   'open': [{'day': 0, 'end': '0200', 'is_overnight': True, 'start': '1600'},
    {'day': 1, 'end': '0200', 'is_overnight': True, 'start': '1600'},
    {'day': 2, 'end': '0200', 'is_overnight': True, 'start': '1600'},
    {'day': 3, 'end': '0200', 'is_overnight': True, 'start': '1600'},
    {'day': 4, 'end': '0200', 'is_overnight': True, 'start': '1600'},
    {'day': 5, 'end': '0200', 'is_overnight': True, 'start': '1600'},
    {'day': 6, 'end': '0200', 'is_overnight': True, 'start': '1600'}]}],
 'id': '71wA42kxcHYwbcwiGvXhNA',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/Pz7mItXPONfJrg4Fu-KW4w/o.jpg',
 'is_claimed': False,
 'is_closed': False,
 'location': {'address1': '3601 W 6th S